In [1]:
import numpy as np
import keras
import pandas as pd
import os
import math
import sys
import soundfile as sf
from sklearn.preprocessing import MinMaxScaler
import random
import tensorflow as tf

import librosa
import matplotlib.pyplot as plt
import pickle

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
    except RuntimeError as e:
        print(e)

2022-05-17 10:35:31.150992: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-17 10:35:31.156903: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-17 10:35:31.465437: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-17 10:35:31.467411: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-17 10:35:31.469252: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

In [3]:
loss_path = "../output/doce/"

In [34]:
def compute_stats(exp_path):
    train_losses = []
    val_losses = []
    for trial in os.listdir(exp_path):
        if os.path.exists(os.path.join(exp_path,trial,"training.log")):
            loss = pd.read_csv(os.path.join(exp_path,trial,"training.log"))
            train_losses.append(min(loss["loss"]))
            val_losses.append(min(loss["val_loss"])) #min within each trial
    return {"min train loss":min(train_losses),"min val loss":min(val_losses)} #min over all trials
        

In [39]:
from collections import defaultdict
#build a huge matrix with feature type, J, Q, min validation loss, min training loss
metrics = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(dict))))
for exp in os.listdir(loss_path):
    if len(exp.split("_"))==7:
        feature_type,J,Q,_,_,act,_ = exp.split("_")
    else:
        feature_type = exp.split("_")[0]+"_"+exp.split("_")[1]
        J,Q,_,_,act,_ = exp.split("_")[2:]
    
    metrics[J][Q][feature_type][act] = compute_stats(os.path.join(loss_path,exp))
    
    print(feature_type,J,Q,act)

scattering_o2 J8 Q16 sigmoid
scattering_o2 J8 Q1 linear
scattering_o2 J10 Q1 sigmoid
scattering_o2 J8 Q16 linear
scattering_o1 J12 Q1 sigmoid
cqt J8 Q16 linear
scattering_o1 J10 Q1 sigmoid
cqt J8 Q1 sigmoid
scattering_o2 J12 Q16 linear
scattering_o2 J8 Q1 sigmoid
scattering_o1 J12 Q16 sigmoid
scattering_o2 J10 Q1 linear
cqt J8 Q1 linear
scattering_o1 J10 Q16 sigmoid
scattering_o2 J10 Q16 linear
scattering_o1 J8 Q16 sigmoid
scattering_o2 J10 Q16 sigmoid
scattering_o1 J8 Q1 linear
scattering_o1 J8 Q1 sigmoid
scattering_o2 J12 Q1 linear
cqt J8 Q16 sigmoid
scattering_o1 J10 Q16 linear
scattering_o1 J10 Q1 linear
scattering_o1 J8 Q16 linear
scattering_o1 J12 Q16 linear
scattering_o2 J12 Q1 sigmoid
scattering_o1 J12 Q1 linear


In [63]:
pd_metrics = pd.DataFrame.from_dict({(i,j,k,m): metrics[i][j][k][m] 
                                    for i in metrics.keys() 
                                    for j in metrics[i].keys()
                                    for k in metrics[i][j].keys()
                                    for m in metrics[i][j][k].keys()
                                    #for n in metrics[i][j][k][m].keys()
                                    },orient="index")

In [64]:
pd_metrics

min train loss  min val loss
J8  Q16 scattering_o2 sigmoid        0.014426      0.024201
                      linear         0.016271      0.028066
        cqt           linear         0.015988      0.024863
                      sigmoid        0.012722      0.020706
        scattering_o1 sigmoid        0.013732      0.023001
                      linear         0.020095      0.029616
    Q1  scattering_o2 linear         0.020334      0.021035
                      sigmoid        0.014363      0.014868
        cqt           sigmoid        0.021478      0.015526
                      linear         0.026583      0.022535
        scattering_o1 linear         0.024019      0.017456
                      sigmoid        0.021518      0.014885
J10 Q1  scattering_o2 sigmoid        0.015232      0.016013
                      linear         0.020266      0.017866
        scattering_o1 sigmoid        0.019775      0.013892
                      linear         0.024462      0.017545
    Q16 scattering_o1 sigmoid        0.014100      0.027496
                      linear         0.016914      0.029809
        scattering_o2 linear         0.016083      0.026013
                      sigmoid        0.010697      0.023077
J12 Q1  scattering_o1 sigmoid        0.021111      0.015485
                      linear         0.026174      0.019309
        scattering_o2 linear         0.019131      0.019892
                      sigmoid        0.015374      0.013984
    Q16 scattering_o2 linear         0.019929      0.038939
        scattering_o1 sigmoid        0.013011      0.025044
                      linear         0.017601      0.029185

In [ ]:
def plot_loss_curve(loss_path,title):
    loss = pd.read_csv(loss_path)
    plt.plot(loss["loss"],label="training loss")
    plt.plot(loss["val_loss"],label="validation loss")
    print("minimum loss",np.min(loss["loss"]),np.min(loss["val_loss"]))
    plt.title(title)
    plt.xlabel("epoch number")
    plt.ylabel("Loss")
    plt.legend()
   

def yield_prediction(test_batches,ckpt_path,model):
    model.load_weights(ckpt_path)
    preds = model.predict(pescador.tuples(test_batches,'input','y'),steps=100,verbose=1)
    return preds

In [12]:

#split up premade features into separate files
feature_path = "/home/han/data/drum_data/han2022features-pkl/scattering_o2/"
for exp in os.listdir(feature_path):
    if "Q-16" in exp and ".pkl" in exp:
        pkl_fl = open(os.path.join(feature_path,exp), 'rb')
        Sy,y = pickle.load(pkl_fl)
        exp_folder = os.path.join(feature_path,exp[:-4])
        os.makedirs(exp_folder,exist_ok=True)
        n_files = Sy.shape[0]
        print(exp,n_files)
        for i in range(n_files):
            file = str(i)+"_"+exp[:-4]+".npy"
            #print(file)
            np.save(os.path.join(exp_folder,file), [Sy[i,:,:],y[i,:]], allow_pickle=True)
            

scattering_o2_fold-test_J-10_Q-16.pkl 10000
scattering_o2_fold-test_J-08_Q-16.pkl 10000
scattering_o2_fold-val_J-10_Q-16.pkl 7776
scattering_o2_fold-val_J-12_Q-16.pkl 7776
scattering_o2_fold-train_J-14_Q-16.pkl 82224
scattering_o2_fold-test_J-12_Q-16.pkl 10000
scattering_o2_fold-train_J-12_Q-16.pkl 82224
scattering_o2_fold-val_J-08_Q-16.pkl 7776
scattering_o2_fold-train_J-08_Q-16.pkl 82224
scattering_o2_fold-train_J-10_Q-16.pkl 82224


In [3]:
Sy = np.ones((100,40,42))
mean = np.mean(Sy,axis=0)
print(mean.shape)

(40, 42)


In [4]:
feature_path = "/home/han/data/drum_data/han2022features-pkl/scattering_o2/"
#save mean and variance per feature for each training set
for exp in os.listdir(feature_path):   
    if "Q-16" in exp and ".pkl" in exp and "train" in exp:
        pkl_fl = open(os.path.join(feature_path,exp), 'rb')
        Sy,y = pickle.load(pkl_fl)
        exp_folder = os.path.join(feature_path,exp[:-4])
        os.makedirs(exp_folder,exist_ok=True)
        s_mean = np.mean(Sy,axis=0)
        s_var = np.var(Sy,axis=0)
        file = exp[:-4]+"_stats.npy"
        np.save(os.path.join(exp_folder,file), [s_mean,s_var], allow_pickle=True)
        